# Solar and Wind Curtailment

Renewables like wind and solar regularly produce energy in excess of demand. 

In order to keep supply and demand balanced on the grid, the result is "curtailment", or purposefully reducing output.

In [1]:
import gridstatus
import pandas as pd
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
def save_figure(fig, name):
    template="plotly_dark"    
    fig.update_layout(template=template)
    
    fig.add_annotation(x=1, y=-.2,
                text="Source: GridStatus.io",
                xref="paper", yref="paper",
                showarrow=False)

    fig.update_layout(height=1200, width=600)
    return fig

In [3]:
iso = gridstatus.CAISO()

In [4]:
df = iso.get_curtailment(start="Jan 1, 2020", end="Nov 30, 2022")

  0%|          | 0/1064 [00:00<?, ?it/s]

In [125]:
df = gridstatus.load_folder("../../../archived_iso_data/caiso/curtailment/", 
                            time_zone=gridstatus.CAISO.default_timezone)

# df_solar = df[df["Fuel Type"] == "Solar"]
# df_wind = df[df["Fuel Type"] == "Wind"]
df["Type"] = (
        df["Curtailment Reason"].str.lower().str.capitalize()
        + " "
        + df["Fuel Type"]
        + " Curtailment (MWh)"
    )
curtailment = df.pivot_table(
        values="Curtailment (MWh)", index="Time", columns="Type"
    ).fillna(
        0
    )

curtailment["Total Solar Curtailment (MWh)"] = curtailment["Local Solar Curtailment (MWh)"] + curtailment["System Solar Curtailment (MWh)"] 
curtailment["Total Wind Curtailment (MWh)"] = curtailment["Local Wind Curtailment (MWh)"] + curtailment["System Wind Curtailment (MWh)"] 
curtailment["Total Curtailment (MWh)"] = curtailment["Total Solar Curtailment (MWh)"] + curtailment["Total Wind Curtailment (MWh)"]
curtailment

100%|██████████| 2332/2332 [00:02<00:00, 891.83it/s]


Type,Local Solar Curtailment (MWh),Local Wind Curtailment (MWh),System Solar Curtailment (MWh),System Wind Curtailment (MWh),Total Solar Curtailment (MWh),Total Wind Curtailment (MWh),Total Curtailment (MWh)
Time,,,,,,,
2016-06-30 00:00:00-07:00,0.0,3.0,0.0,0.0,0.0,3.0,3.0
2016-06-30 01:00:00-07:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-06-30 06:00:00-07:00,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2016-06-30 10:00:00-07:00,0.0,20.0,0.0,0.0,0.0,20.0,20.0
2016-06-30 13:00:00-07:00,0.0,13.0,0.0,0.0,0.0,13.0,13.0
...,...,...,...,...,...,...,...
2022-11-17 12:00:00-08:00,95.0,0.0,0.0,0.0,95.0,0.0,95.0
2022-11-17 13:00:00-08:00,143.0,0.0,0.0,0.0,143.0,0.0,143.0
2022-11-17 14:00:00-08:00,18.0,0.0,0.0,0.0,18.0,0.0,18.0


In [126]:
mix = gridstatus.load_folder("../../../archived_iso_data/caiso/fuel_mix/", 
                            time_zone=gridstatus.CAISO.default_timezone).set_index("Time")

load = gridstatus.load_folder("../../../archived_iso_data/caiso/load/",
                                time_zone=gridstatus.CAISO.default_timezone).set_index("Time")

mix = pd.concat([mix, load], axis=1)                                
mix = mix.resample("1H").mean()

100%|██████████| 1676/1676 [00:01<00:00, 915.26it/s] 


In [127]:
data = pd.concat([mix, curtailment], axis=1).dropna()

In [62]:
monthly = curtailment.resample("M").sum()
monthly["Month"] = monthly.index.month
monthly["Year"] = monthly.index.year

fig = px.bar(monthly, 
    x=monthly.index, y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Monthly Solar and Wind Curtailment in CAISO (MWh)",)
# legend upper left corner
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="left",
    x=0,
    title_text=None
))
fig.update_yaxes(title_text="Curtailment (MWh)")
save_figure(fig, "monthly_curtailment")

In [61]:
monthly_avg = curtailment.groupby("Month").mean()
monthly_avg.index = monthly_avg.index.map(lambda x: pd.Timestamp(year=2020, month=x, day=1).strftime("%B"))

fig = px.bar(monthly_avg,
    x=monthly_avg.index, y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Average Solar and Wind Curtailment in CAISO (MWh) 2016 - 2022",)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=.95,
    xanchor="right",
    x=1,
    title_text=None
))
fig.update_yaxes(title_text="Curtailment (MWh)")
save_figure(fig, "monthly_avg_curtailment")


In [116]:
curtailment["Year"] = curtailment.index.year
yearly_sum = curtailment.groupby("Year").sum()
index = yearly_sum.index.astype(str).tolist()
index[-1] = "2022 YTD"
yearly_sum.index = index

fig = px.bar(yearly_sum,
    x=yearly_sum.index, y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Total Solar and Wind Curtailment in CAISO (MWh)",)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=.9,
    xanchor="left",
    x=0,
    title_text=None
))
fig.update_yaxes(title_text="Curtailment (MWh)")
fig.update_xaxes(title_text="Year")
save_figure(fig, "yearly_sum_curtailment")


In [68]:
curtailment["Hour"] = curtailment.index.hour

hourly_avg = curtailment.groupby("Hour").mean()

fig = px.bar(hourly_avg,
    x=hourly_avg.index, y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Average Solar and Wind Curtailment in CAISO (MWh) 2016 - 2022",)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=.95,
    xanchor="right",
    x=1,
    title_text=None
))
fig.update_yaxes(title_text="Curtailment (MWh)")
save_figure(fig, "hourly_avg_curtailment")


In [100]:
data

,Solar,Wind,Geothermal,Biomass,Biogas,Small Hydro,Coal,Nuclear,Natural Gas,Large Hydro,...,Imports,Other,Load,Local Solar Curtailment (MWh),Local Wind Curtailment (MWh),System Solar Curtailment (MWh),System Wind Curtailment (MWh),Total Solar Curtailment (MWh),Total Wind Curtailment (MWh),Hour
Time,,,,,,,,,,,,,,,,,,,,,
2018-04-10 04:00:00-07:00,0.000000,858.666667,726.333333,294.083333,234.166667,402.416667,-5.833333,1616.750000,7171.000000,3126.666667,...,6326.250000,0.0,20114.500000,0.0,0.0,0.0,5.0,0.0,5.0,4
2018-04-10 05:00:00-07:00,0.000000,753.333333,727.166667,295.083333,234.500000,435.416667,-5.333333,1617.500000,7228.583333,3545.166667,...,6673.916667,0.0,21254.833333,0.0,7.0,0.0,0.0,0.0,7.0,5
2018-04-10 06:00:00-07:00,95.250000,794.833333,729.250000,297.916667,232.833333,500.750000,-6.000000,1616.333333,8211.250000,3777.833333,...,7150.083333,0.0,23215.250000,0.0,6.0,0.0,0.0,0.0,6.0,6
2018-04-10 07:00:00-07:00,2150.333333,849.666667,730.083333,299.416667,233.333333,550.083333,-6.583333,1615.333333,7832.416667,3645.416667,...,6461.166667,0.0,24219.000000,0.0,6.0,0.0,0.0,0.0,6.0,7
2018-04-10 08:00:00-07:00,5977.750000,1032.750000,728.250000,298.916667,232.833333,477.166667,-5.000000,1615.500000,6595.166667,3165.750000,...,4535.666667,0.0,24307.833333,5.0,6.0,0.0,0.0,5.0,6.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-10 11:00:00-08:00,11545.666667,1486.750000,834.000000,241.916667,206.500000,112.750000,2.833333,1133.166667,8403.916667,434.833333,...,31.833333,0.0,19615.416667,36.0,0.0,3.0,0.0,39.0,0.0,11
2022-11-10 12:00:00-08:00,11624.416667,1418.000000,851.916667,239.750000,210.916667,114.416667,2.333333,1132.083333,8313.416667,435.416667,...,104.083333,0.0,19562.166667,35.0,0.0,0.0,0.0,35.0,0.0,12
2022-11-10 13:00:00-08:00,11717.083333,1516.666667,850.250000,239.833333,210.583333,113.166667,2.333333,1132.000000,7827.416667,479.500000,...,-104.916667,0.0,20162.166667,44.0,0.0,6.0,0.0,50.0,0.0,13


In [147]:
data["Hour"] = data.index.hour

hourly_avg = data.groupby("Hour").mean()

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=hourly_avg.index, y=hourly_avg["Total Solar Curtailment (MWh)"], name="Total Solar Curtailment"),
    secondary_y=False,
)

fig.add_bar(x=hourly_avg.index,
            y=hourly_avg["Total Wind Curtailment (MWh)"], name="Total Wind Curtailment")
fig.update_layout(barmode='stack')


fig.add_trace(
    go.Scatter(x=hourly_avg.index, y=hourly_avg["Load"], name="Load"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(title_text="Load vs Curtailment CAISO (2018 - 2022 YTD)")

# Set x-axis title
fig.update_xaxes(title_text="Hour")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Curtailment (MWh)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Load (MWh)</b>", secondary_y=True)
# horizontal legend
fig.update_layout(legend=dict(yanchor="top", y=1.05, xanchor="left", x=0))
save_figure(fig, "load_vs_curtailment")

In [144]:
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Bar(x=hourly_avg.index, y=hourly_avg["Batteries"], name="Battery (negative means charging)"),
)

# fig.add_bar(x=hourly_avg.index,
#             y=hourly_avg["Total Wind Curtailment (MWh)"], name="Total Wind Curtailment (MWh)")
# fig.update_layout(barmode='stack')


fig.add_trace(
    go.Scatter(x=hourly_avg.index, y=hourly_avg["Total Curtailment (MWh)"], name="Total Curtailment (MWh)"),
)

# Add figure title
fig.update_layout(title_text="Battery Dispatch vs Curtailment CAISO (2018 - 2022 YTD)")

# Set x-axis title
fig.update_xaxes(title_text="Hour")

# Set y-axes titles
fig.update_yaxes(title_text="<b>MWh</b>")
# horizontal legend
# legend font size
fig.update_layout(legend=dict(yanchor="top", y=1, xanchor="left", x=-.05))
save_figure(fig, "battery_vs_curtailment")

In [73]:
data

,Solar,Wind,Geothermal,Biomass,Biogas,Small Hydro,Coal,Nuclear,Natural Gas,Large Hydro,Batteries,Imports,Other,Local Solar Curtailment (MWh),Local Wind Curtailment (MWh),System Solar Curtailment (MWh),System Wind Curtailment (MWh),Total Solar Curtailment (MWh),Total Wind Curtailment (MWh),Hour
Time,,,,,,,,,,,,,,,,,,,,
2018-04-10 04:00:00-07:00,0.000000,858.666667,726.333333,294.083333,234.166667,402.416667,-5.833333,1616.750000,7171.000000,3126.666667,-5.666667,6326.250000,0.0,0.0,0.0,0.0,5.0,0.0,5.0,4.0
2018-04-10 05:00:00-07:00,0.000000,753.333333,727.166667,295.083333,234.500000,435.416667,-5.333333,1617.500000,7228.583333,3545.166667,-9.416667,6673.916667,0.0,0.0,7.0,0.0,0.0,0.0,7.0,5.0
2018-04-10 06:00:00-07:00,95.250000,794.833333,729.250000,297.916667,232.833333,500.750000,-6.000000,1616.333333,8211.250000,3777.833333,-0.666667,7150.083333,0.0,0.0,6.0,0.0,0.0,0.0,6.0,6.0
2018-04-10 07:00:00-07:00,2150.333333,849.666667,730.083333,299.416667,233.333333,550.083333,-6.583333,1615.333333,7832.416667,3645.416667,4.750000,6461.166667,0.0,0.0,6.0,0.0,0.0,0.0,6.0,7.0
2018-04-10 08:00:00-07:00,5977.750000,1032.750000,728.250000,298.916667,232.833333,477.166667,-5.000000,1615.500000,6595.166667,3165.750000,0.000000,4535.666667,0.0,5.0,6.0,0.0,0.0,5.0,6.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-10 11:00:00-08:00,11545.666667,1486.750000,834.000000,241.916667,206.500000,112.750000,2.833333,1133.166667,8403.916667,434.833333,-1397.416667,31.833333,0.0,36.0,0.0,3.0,0.0,39.0,0.0,11.0
2022-11-10 12:00:00-08:00,11624.416667,1418.000000,851.916667,239.750000,210.916667,114.416667,2.333333,1132.083333,8313.416667,435.416667,-1449.333333,104.083333,0.0,35.0,0.0,0.0,0.0,35.0,0.0,12.0
2022-11-10 13:00:00-08:00,11717.083333,1516.666667,850.250000,239.833333,210.583333,113.166667,2.333333,1132.000000,7827.416667,479.500000,-852.583333,-104.916667,0.0,44.0,0.0,6.0,0.0,50.0,0.0,13.0


In [69]:
data["Hour"] = data.index.hour

fig = px.bar(

,Solar,Wind,Geothermal,Biomass,Biogas,Small Hydro,Coal,Nuclear,Natural Gas,Large Hydro,Batteries,Imports,Other,Local Solar Curtailment (MWh),Local Wind Curtailment (MWh),System Solar Curtailment (MWh),System Wind Curtailment (MWh),Total Solar Curtailment (MWh),Total Wind Curtailment (MWh)
Time,,,,,,,,,,,,,,,,,,,
2018-04-10 04:00:00-07:00,0.0,817.0,726.0,289.0,234.0,373.0,-7.0,1619.0,7738.0,2768.0,-19.0,6282.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0
2018-04-10 05:00:00-07:00,0.0,860.0,727.0,294.0,235.0,410.0,-5.0,1619.0,6683.0,3423.0,-22.0,6501.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0
2018-04-10 06:00:00-07:00,0.0,711.0,728.0,298.0,234.0,484.0,-6.0,1619.0,7638.0,3621.0,-1.0,7043.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0
2018-04-10 07:00:00-07:00,634.0,844.0,729.0,297.0,233.0,547.0,-7.0,1617.0,8259.0,3803.0,0.0,6994.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0
2018-04-10 08:00:00-07:00,4326.0,979.0,730.0,299.0,234.0,532.0,-5.0,1617.0,6883.0,3507.0,-5.0,5282.0,0.0,5.0,6.0,0.0,0.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-10 11:00:00-08:00,11502.0,1451.0,841.0,243.0,207.0,113.0,3.0,1133.0,8589.0,430.0,-1505.0,200.0,0.0,36.0,0.0,3.0,0.0,39.0,0.0
2022-11-10 12:00:00-08:00,11583.0,1458.0,852.0,240.0,209.0,111.0,2.0,1133.0,8224.0,435.0,-1263.0,-172.0,0.0,35.0,0.0,0.0,0.0,35.0,0.0
2022-11-10 13:00:00-08:00,11750.0,1427.0,852.0,237.0,212.0,114.0,2.0,1132.0,8591.0,433.0,-1614.0,291.0,0.0,44.0,0.0,6.0,0.0,50.0,0.0


In [77]:
fig = px.scatter(data, y="Total Solar Curtailment (MWh)", x="Natural Gas", title="Hourly Solar Curtailment vs. Natural Gas Generation", opacity=0.5)
fig.update_xaxes(title_text="Natural Gas Generation (MWh)")
# x axis range minumum 0
fig.update_xaxes(range=[0, 27500])

save_figure(fig, "solar_curtailment_vs_gas")

In [36]:
monthly = data.resample("M").sum()[:-1]
monthly["Percent Solar Curtailment"] = (monthly["Total Solar Curtailment (MWh)"] / (np.maximum(monthly["Solar"],0) + monthly["Total Solar Curtailment (MWh)"])).fillna(0)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=monthly.index, y=monthly["Percent Solar Curtailment"], name="Percent Solar Curtailment"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=monthly.index, y=monthly["Solar"], name="Solar Production"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(title_text="Solar Curtailment")

# Set x-axis title
fig.update_xaxes(title_text="Time")

# fig = px.bar(daily, x=daily.index, y=daily["Percent Solar Curtailment"], title="Monthly Percent Solar Curtailment in CAISO")
save_figure(fig, "percent_curtailed_solar")

In [25]:
daily = data.resample("M").sum()
daily["Percent Wind Curtailment"] = (daily["Total Wind Curtailment (MWh)"] / (np.maximum(daily["Wind"],0) + daily["Total Wind Curtailment (MWh)"])).fillna(0)

fig = px.bar(daily, x=daily.index, y=daily["Percent Wind Curtailment"], title="Monthly Percent Wind Curtailment in CAISO")
save_figure(fig, "percent_curtailed_wind")

In [ ]:
hourly_with_price = pd.read_csv("../../../archived_iso_data/hourly_with_price.csv")
hourly_with_price["Time"] = pd.to_datetime(hourly_with_price["Time"], utc=True).dt.tz_convert(gridstatus.CAISO.default_timezone)

In [ ]:
mix_hourly = mix.set_index("Time").resample("H").mean().reset_index()

solar_curtailment_hourly = df_solar.set_index("Time")[["Curtailment (MWh)"]].resample("H").mean().fillna(0).reset_index().rename(columns={"Curtailment (MWh)": "Solar Curtailment (MWh)"})
wind_curtailment_hourly = df_wind.set_index("Time")[["Curtailment (MWh)"]].resample("H").mean().fillna(0).reset_index().rename(columns={"Curtailment (MWh)": "Wind Curtailment (MWh)"})

mix_w_curtailment = mix_hourly.merge(solar_curtailment_hourly, on="Time").merge(wind_curtailment_hourly, on="Time")
mix_w_curtailment["Total Curtailment (MWh)"] = mix_w_curtailment["Solar Curtailment (MWh)"] + mix_w_curtailment["Wind Curtailment (MWh)"]
mix_w_curtailment["Has Curtailment"] = mix_w_curtailment["Total Curtailment (MWh)"] > 0
mix_w_curtailment["Year"] = mix_w_curtailment["Time"].dt.year.replace(2022, "2022 YTD").astype(str)
mix_w_curtailment["Hour"] = mix_w_curtailment["Time"].dt.hour
mix_w_curtailment["Month"] = mix_w_curtailment["Time"].dt.month_name()
mix_w_curtailment["Date"] = mix_w_curtailment["Time"].dt.strftime("%Y%m%d")

In [ ]:
fig = px.bar(
    mix_w_curtailment.groupby(["Year"]).sum().reset_index(),
    x="Year", 
    y=["Solar Curtailment (MWh)", "Wind Curtailment (MWh)"], 
    title="CAISO Annual Renewable Curtailment")
fig.update_xaxes(nticks=6, title="Year")
fig.update_yaxes(title="Curtailment (MWh)")
fig.update_layout(legend_title_text='Curtailment Type')
save_figure(fig, "caiso/curtailment_annual")

In [ ]:
px.bar(mix_w_curtailment.groupby("Year")[["Solar", "Wind"]].sum().reset_index(), x="Year", y=["Solar", "Wind"])

In [ ]:
monthly = mix_w_curtailment.set_index("Time").resample("M").sum().reset_index()
monthly["Year"] = monthly["Time"].dt.year
monthly["Month"] = monthly["Time"].dt.month
fig = px.bar(
    monthly, 
    x="Time", 
    y="Total Curtailment (MWh)", 
    title="Monthly Curtailment - CAISO (2019 - 2022)")
fig.update_traces(marker_color=monthly["Year"])
fig.update_xaxes(nticks=4)
save_figure(fig, "caiso/curtailment_monthly")

In [ ]:
monthly = mix_w_curtailment.set_index("Time").resample("M").sum().reset_index()
monthly["Year"] = monthly["Time"].dt.year
monthly["Month"] = monthly["Time"].dt.month
fig = px.bar(
    monthly, 
    x="Time", 
    y="Solar Curtailment (MWh)", 
    title="Monthly Solar Curtailment - CAISO (2019 - 2022)")
fig.update_traces(marker_color=monthly["Year"])
fig.update_xaxes(nticks=4)
save_figure(fig, "caiso/solar_curtailment_monthly")

In [ ]:
fig = px.bar(
    monthly, 
    x="Time", 
    y="Wind Curtailment (MWh)", 
    title="Wind Solar Curtailment - CAISO (2019 - 2022)")
fig.update_traces(marker_color=monthly["Year"])
fig.update_xaxes(nticks=4)
save_figure(fig, "caiso/wind_curtailment_monthly")

In [ ]:
fig = px.line(
    mix_w_curtailment.groupby(["Month", "Year"]).sum().reset_index(),
    x="Month", 
    y="Solar Curtailment (MWh)", 
    color="Year", 
    title="Solar Curtailment - CAISO (2019 - 2022)")
fig.update_xaxes(nticks=6)
save_figure(fig, "caiso/solar_curtailment_annual")

In [ ]:
fig = px.line(
    mix_w_curtailment.groupby(["Month", "Year"]).sum().reset_index(),
    x="Month", 
    y="Wind Curtailment (MWh)", 
    color="Year", 
    title="Wind Curtailment - CAISO (2019 - 2022)")
fig.update_xaxes(nticks=6)
save_figure(fig, "caiso/wind_curtailment_annual")

In [ ]:
fig = px.line(mix_w_curtailment.groupby(["Month", "Hour"]).sum().reset_index(), 
              x="Hour",
              y="Solar Curtailment (MWh)", 
              color="Month", 
              title="Average Solar Curtailment by Hour - CAISO (2019 - 2022)")

save_figure(fig, "caiso/solar_curtailment_by_hour")

In [ ]:
fig = px.line(mix_w_curtailment.groupby(["Month", "Hour"]).sum().reset_index(), 
              x="Hour",
              y="Wind Curtailment (MWh)", 
              color="Month", 
              title="Average Wind Curtailment by Hour - CAISO (2019 - 2022)")

save_figure(fig, "caiso/wind_curtailment_by_hour")

In [ ]:
# 2 = mix_w_curtailment[(mix_w_curtailment["Month"] == "April") &  (mix_w_curtailment["Year"] == "2022 YTD")]
fig = px.line(mix_w_curtailment[mix_w_curtailment["Time"].dt.year > 2021], 
              x="Hour",
              y="Batteries", 
              color="Date", 
              title="Battery Dispatch Profiles - CAISO (2021 - 2022 YTD)")
fig.update_traces(line_color='rgba(155, 155, 155, 0.1)')
fig.update_yaxes(title="Batteries (MW)")
x_pos = 2
y_pos = 300
fig.add_annotation(x=0.005, y=.5,
                   xref="paper", yref="paper",
            text="Discharging",
            showarrow=False)

fig.add_annotation(x=0.005, y=.35,
                   xref="paper", yref="paper",
            text="Charging   ",
            showarrow=False)

fig.update_layout(showlegend=False)
save_figure(fig, "caiso/battery_profiles")
fig

In [ ]:
discharge = mix[mix["Batteries"] > 0]
d = discharge.set_index("Time").resample("H").mean().resample("M").sum().reset_index()
d["Year"] = d["Time"].dt.year
fig = px.bar(
    d, 
    x="Time", y="Batteries",
    title="Monthly Battery Discharge - CASIO")
fig.update_traces(marker_color=d["Year"])
fig.update_yaxes(title="Battery Discharge (KWh)")
save_figure(fig, "caiso/monthly_battery_discharge")

In [ ]:
hourly_with_price["Battery Revenue"] = hourly_with_price["LMP"] * hourly_with_price["Batteries"]
d = hourly_with_price.set_index("Time").resample("M").sum().reset_index()
d["Year"] = d["Time"].dt.year
fig = px.bar(d[:-1], 
        x="Time", 
        y="Battery Revenue",
        title="Battery Profits by Month - CAISO <br><sub>Estimated using NP-15 DAM LMPs</sub>"
        )
fig.update_traces(marker_color=d["Year"])
fig.update_yaxes(title="Battery Profits (USD)")
save_figure(fig, "caiso/battery_revenue")

In [ ]:
px.line(
    df_solar.set_index("Time")[["Curtailment (MWh)"]].resample("H").mean().fillna(0).reset_index(),
    x="Time", 
    y="Curtailment (MWh)")

In [ ]:
px.line(mix_w_curtailment.set_index("Time").resample("W").mean().reset_index(), x="Time", y=["Batteries", "Solar", "Solar Curtailment (MWh)"])

In [ ]:
fig = px.line(mix, x="Time", y="Batteries", title="Hourly Battery Usage")

x_pos = mix["Time"].min() + pd.Timedelta(days=95)
y_pos = 300
fig.add_annotation(x=x_pos, y=y_pos,
            text="Discharging ↑",
            showarrow=False)

fig.add_annotation(x=x_pos, y=-y_pos,
            text="Charging   ",
            showarrow=False)

fig

In [ ]:
mix_w_curtailment["Solar Curtailment (MWh)"].describe()

In [ ]:
mix

In [ ]:
d = mix_w_curtailment[mix_w_curtailment["Natural Gas"] > 0]
fig = px.histogram(
    d,
    x="Natural Gas",
    color="Has Solar Curtailment",
    histnorm='percent',
    barmode='overlay',
    title="Distribution of Natural Gas with and without Curtailment",
    template="seaborn")

In [ ]:
new_record_solar = mix_w_curtailment[mix_w_curtailment["Solar Curtailment (MWh)"].cummax() == mix_w_curtailment["Solar Curtailment (MWh)"]]
new_record_solar["Time"].diff()

In [ ]:
new_record_batteries = mix[(mix["Batteries"].cummin() == mix["Batteries"])] # | (mix["Batteries"].cummin() == mix["Batteries"])
px.line(new_record_batteries, x="Time", y="Batteries")


In [ ]:
# amount of curtailment
px.histogram(df_solar, x="Curtailment (MWh)")